# Test Diff

In [20]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, Incident, URL, EmailMessage, ObservedData
)
from stixorm.module.definitions.os_threat import (
    EventCoreExt, Event, SocialMediaContact, ContactNumber, IncidentCoreExt, TaskCoreExt,
    Task, SightingEvidence, Sequence, SequenceExt, AnecdoteExt, Anecdote,
    SightingAnecdote, SightingAlert, SightingContext, SightingExclusion,
    SightingEnrichment, SightingHunt, SightingFramework, SightingExternal
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
from deepdiff import DeepDiff, parse_path
from pprint import pprint
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
import json

# Load Context Memory

In [21]:

from Utilities.local_make_general import invoke_context_get_block, invoke_context_save_block
# 1. Load the Context
Type_Refinery_Context = invoke_context_get_block()#
# 2. Setup the  TR User Context
#
local = Type_Refinery_Context["local"]
local_context = local["context"]
me = local_context["me"]
team = local_context["team"]
company = local_context["company"]
systems = local_context["systems"]
assets = local_context["assets"]
#
# 3. Setup the Incident Context
#
local_incident = local["incident"]
sequence_start_objs = local_incident["sequence_start_objs"]
sequence_objs = local_incident["sequence_objs"]
task_objs = local_incident["task_objs"]
event_objs = local_incident["event_objs"]
impact_objs = local_incident["impact_objs"]
other_object_objs = local_incident["other_object_objs"]
incident_obj = local_incident["incident_obj"]

# Lets run some tests

In [22]:
A = {0, 2, 4, 6, 8}
B = {1, 2, 3, 4, 5}
C = A - B
print(f"Delete -> Things in old, not in new ->{C}")
D = B - A
print(f"Add -> Things in new, not in old ->{D}")
E = A & B
print(f"Intersections -> Things which may have changed ->{E}")


Delete -> Things in old, not in new ->{0, 8, 6}
Add -> Things in new, not in old ->{1, 3, 5}
Intersections -> Things which may have changed ->{2, 4}


In [23]:
from datetime import datetime
created ="2024-01-24T12:38:07.927748Z"
returned = "2024-01-24T12:38:07.927Z"
cr_date = datetime.strptime(created, "%Y-%m-%dT%H:%M:%S.%fZ")
ret_date = datetime.strptime(returned, "%Y-%m-%dT%H:%M:%S.%fZ")
print(type(cr_date))
print(cr_date)
cr_date_String = str(cr_date.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3])
print(cr_date_String)
print(cr_date.microsecond)
microsecs = ret_date.microsecond
print(type(microsecs))
milisecs = (round(microsecs/1000))
print(milisecs)
dt = created.split('.')
print(dt)
actual = dt[0] + "." + str(milisecs) + "Z"
print(actual)




<class 'datetime.datetime'>
2024-01-24 12:38:07.927748
2024-01-24T12:38:07.927
927748
<class 'int'>
927
['2024-01-24T12:38:07', '927748Z']
2024-01-24T12:38:07.927Z


In [28]:
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from Utilities.local_make_sdo import invoke_make_identity_block
from Utilities.util import emulate_ports, unwind_ports, conv
connection = {
    "uri": "localhost",
    "port": "1729",
    "database": "stix_test",
    "user": None,
    "password": None
}

import_type = import_type_factory.get_all_imports()
company_ident_path = "SDO/Identity/identity_TR_user_company.json"
company_results_path = "step0/employer"
#
# MAke the Company object nd add it to the bundle_list
#
comp_ident_list = invoke_make_identity_block(company_ident_path, company_results_path)
# 2. add the record to the in-session bundles and context lists
local_context["company"] = local_context["company"] + comp_ident_list
# recover the threat email object in dict form as a global variable for later
comp_ident = conv(comp_ident_list[0])

typedb_sink = TypeDBSink(connection=connection, clear=True, import_type=import_type)
typedb_source = TypeDBSource(connection=connection,)
results_raw = typedb_sink.add(comp_ident)
ret_ident = typedb_source.get(comp_ident["id"])
print(ret_ident)

{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--c90a815e-46ee-4e05-9a6e-20daf7b5348d",
    "created": "2024-01-27T02:28:18.780Z",
    "modified": "2024-01-27T02:28:18.780Z",
    "name": "Example Company",
    "description": "An Example Organisation",
    "identity_class": "organization",
    "sectors": [
        "technology"
    ]
}
{"type": "identity", "spec_version": "2.1", "id": "identity--c90a815e-46ee-4e05-9a6e-20daf7b5348d", "created": "2024-01-26T15:28:18.780Z", "modified": "2024-01-26T15:28:18.780Z", "name": "Example Company", "description": "An Example Organisation", "identity_class": "organization", "sectors": ["technology"]}


In [3]:
test = "20.20-10-19T01:01:.01Z"
ret = test.find(".")
print(ret)

2
